In [1]:
# Import Libraries
import rospy
import actionlib.msg
import assignment_2_2023.msg
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import PlanningActionGoal
from assignment_2_2023.msg import PlanningActionResult
from actionlib_msgs.msg import GoalStatus

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation

In [3]:
rospy.init_node('action_client') 

In [4]:
x_input_position = widgets.FloatText(value=7, description ="X_GOAL",disable=False)
y_input_position = widgets.FloatText(value=7, description ="Y_GOAL",disable=False)
button= widgets.Button(description="SEND")
cancel_button= widgets.Button(description="CANCEL")
resume_button= widgets.Button(description="RESUME")

In [5]:
action_client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)

def action(e):

    global action_client,x_input_position,y_input_position
    
    action_client.wait_for_server()           # wait for the server to be started
    status_goal = False


    # Convert the data type of the numbers from string to float
    x_float = float(x_input_position.value)
    y_float = float(y_input_position.value)
            
    # creat the goal to send to the server
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x_float
    goal.target_pose.pose.position.y = y_float
    action_client.send_goal(goal)                      # send the goal data to the action server
    status_goal = True

In [6]:
def cancel(e):
  
    # create the action client
    global action_client
    
    action_client.wait_for_server()  # wait for the server to be started
    action_client.cancel_goal()      # cancel the goal
    status_goal = False     

In [7]:
button.on_click(action)
cancel_button.on_click(cancel)
resume_button.on_click(action)

In [8]:
display(button,resume_button,cancel_button,x_input_position,y_input_position)


Button(description='SEND', style=ButtonStyle())

Button(description='RESUME', style=ButtonStyle())

Button(description='CANCEL', style=ButtonStyle())

FloatText(value=7.0, description='X_GOAL')

FloatText(value=7.0, description='Y_GOAL')

In [9]:
class Visualiser:

    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []
        self.x_target, self.y_target= [] , []
        
        # to draw a bar graph, we need initialized 
        self.reached = 0
        self.cancelled = 0
        self.fig1, self.ax1 = plt.subplots()
        self.labels = ['Reached', 'Not Reached']
        self.values = [self.reached, self.cancelled]
        
        
    def plot_init(self):
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)
        self.ax.plot(self.x_target, self.y_target, 'gx', label='Target Point')
        self.ax.plot(self.x_data,self.y_data, 'ro',label='Robot Path')
        self.ax.legend()
        return self.ln,
    
    def bar_init(self):
        self.ax1.bar(self.labels, self.values)
        self.ax1.set_ylabel('Count')
        self.ax1.set_title('Target Status')
        
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    def target(self,msg):
        self.x_target = msg.goal.target_pose.pose.position.x
        self.y_target = msg.goal.target_pose.pose.position.y     
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.ax.plot(self.x_target,self.y_target, 'gx')
        return self.ln,77
    def reached_callback(self, message):    
        if message.status.status == actionlib.GoalStatus.SUCCEEDED:
            self.reached +=1
        elif message.status.status == actionlib.GoalStatus.PREEMPTED:
            self.cancelled +=1
    def update_bar(self,frame):
        self.values = [self.reached, self.cancelled]
        self.ax1.bar(self.labels, self.values, color = 'purple')
        self.ax1.set_ylim([0, max(vis.reached, vis.cancelled) + 1])
        

In [10]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
tar = rospy.Subscriber('/reaching_goal/goal', PlanningActionGoal, vis.target)
reach = rospy.Subscriber('/reaching_goal/result',PlanningActionResult, vis.reached_callback)
ani1 = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
ani2 = FuncAnimation(vis.fig1, vis.update_bar, init_func=vis.bar_init)
plt.show(block=True) 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …